In [14]:
import sys
sys.path.append(r'C:\Users\patry\NLP_app')

from image_text_generator import ImageTextGenerator

generator = ImageTextGenerator(model_name="flax-community/papuGaPT2", fonts_folder=r"C:\Users\patry\Desktop\wypakwoane", random_state=42)
df = generator.generate_data()


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generati

In [15]:
text_bag = df['text'].explode().tolist()

In [16]:
from tokenizers import Tokenizer, models, normalizers, pre_tokenizers, trainers, processors, decoders
from tokenizers.normalizers import Sequence, NFD, Lowercase, StripAccents, Replace
from tokenizers.pre_tokenizers import Whitespace, Punctuation
from tokenizers.trainers import WordPieceTrainer
from tokenizers.processors import TemplateProcessing
import tokenizers
import os

# Tworzenie modelu tokenizacji
tokenizer = Tokenizer(models.WordPiece(unk_token='[UNK]'))

# Ustawienie normalizera z poprawnym użyciem tokenizers.Regex
tokenizer.normalizer = Sequence([
    # Usuwanie znaków specjalnych, ale nie białych znaków
    Replace(tokenizers.Regex(r"[\p{Other}&&[^\n\t\r]]"), ""),  
    Lowercase(),
    NFD(),
    StripAccents()
])

# Ustawienie pre-tokenizera
tokenizer.pre_tokenizer = pre_tokenizers.Sequence([
    Whitespace(),  # Dzieli po białych znakach
    Punctuation()  # Dzieli znaki interpunkcyjne
])

# Ustawienie trenera
special_tokens = ["[UNK]", "[PAD]", "[CLS]", "[SEP]", "[MASK]"]
trainer = WordPieceTrainer(vocab_size=1702, special_tokens=special_tokens)

# Trening tokenizera
tokenizer.train_from_iterator(text_bag, trainer=trainer)

# Pobieranie identyfikatorów specjalnych tokenów
cls_token_id = tokenizer.token_to_id("[CLS]")
sep_token_id = tokenizer.token_to_id("[SEP]")

# Ustawienie post-processora
tokenizer.post_processor = TemplateProcessing(
    single = f"[CLS]:0 $A:0 [SEP]:0",
    pair = f"[CLS]:0 $A:0 [SEP]:0 $B:1 [SEP]:1",
    special_tokens = [("[CLS]", cls_token_id), ("[SEP]", sep_token_id)]
)

# Ustawienie dekodera
tokenizer.decoder = decoders.WordPiece(prefix="##")

# Testowanie tokenizera
encoded = tokenizer.encode("put down a resolution")
print(encoded.tokens)  # Powinno wyświetlić tokeny

# Zapisanie tokenizera do pliku w podanej lokalizacji
save_path = "C:/Users/patry/NLP_app/tokenizer_for_GAN"
os.makedirs(os.path.dirname(save_path), exist_ok=True)  # Tworzy katalog, jeśli nie istnieje
tokenizer.save(save_path)

print(f"Tokenizer został zapisany w lokalizacji: {save_path}")


['[CLS]', 'put', 'down', 'a', 'res', '##ol', '##ution', '[SEP]']
Tokenizer został zapisany w lokalizacji: C:/Users/patry/NLP_app/tokenizer_for_GAN


In [17]:
from tokenizers import Tokenizer

# Ścieżka do pliku z zapisanym tokenizatorem
load_path = "C:/Users/patry/NLP_app/tokenizer_for_GAN"

# Importowanie tokenizatora z pliku
tokenizer = Tokenizer.from_file(load_path)

# Testowanie zaimportowanego tokenizatora
encoded = tokenizer.encode("put down a resolution")
print(encoded.tokens)  # Wyświetla zakodowane tokeny

['[CLS]', 'put', 'down', 'a', 'res', '##ol', '##ution', '[SEP]']


In [18]:
tokenizer.enable_padding(pad_id=tokenizer.token_to_id("[PAD]"), pad_token="[PAD]", length=100)
#tokenizer.enable_truncation(max_length=10)  # Maksymalna długość sekwencji ustawiona na 10

# Testowanie tokenizera na różnych długościach tekstu
encoded_1 = tokenizer.encode("put down a resolution")
encoded_2 = tokenizer.encode("this is a longer example sentence that needs truncation")
encoded_3 = tokenizer.encode("short")

print(f"Tokeny (put down a resolution): {encoded_1.tokens}")
print(f"Tokeny (this is a longer example...): {encoded_2.tokens}")
print(f"Tokeny (short): {encoded_3.tokens}")

print(f"Długość (put down a resolution): {len(encoded_1.tokens)}")
print(f"Długość (this is a longer example...): {len(encoded_2.tokens)}")
print(f"Długość (short): {len(encoded_3.tokens)}")

Tokeny (put down a resolution): ['[CLS]', 'put', 'down', 'a', 'res', '##ol', '##ution', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]']
Tokeny (this is a longer example...): ['[CLS]', 'this', 'is', 'a', 'long', 

In [20]:
encoded_2.ids

[2,
 205,
 152,
 31,
 583,
 100,
 212,
 605,
 132,
 918,
 290,
 141,
 594,
 74,
 243,
 298,
 75,
 177,
 3,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1]